# 훈련 환경 설정

- 우선, 등록한 템플릿으로 스팟 인스턴스를 생성합니다.
- [트랙 정보](https://github.com/aws-deepracer-community/deepracer-race-data/tree/main/raw_data/tracks)
- [하이퍼파라미터 설정 : Wiki](https://wiki.deepracing.io/Customise_Local_Training)

## 1. Worker 1개

### 루트 사용자 전환
  
```bash
sudo su

```

### 버킷 설정
  
```bash
# 작업폴더이동
cd ~/deepracer-for-cloud/

# dr-* 명령어 가져오기
source bin/activate.sh

############## 주요 환경변수 설정 ##############
# S3 버킷 - 로컬
DR_LOCAL_S3_BUCKET=drfc-....-local

# S3 버킷 - 업로드
DR_UPLOAD_S3_BUCKET=drfc-....-upload

# 버킷 생성
aws s3 mb s3://${DR_LOCAL_S3_BUCKET} --region us-east-1
aws s3 mb s3://${DR_UPLOAD_S3_BUCKET} --region us-east-1

# 환경변수 등록
sed -i "s/\(^DR_LOCAL_S3_BUCKET=\)\(.*\)/\1$DR_LOCAL_S3_BUCKET/" system.env
sed -i "s/\(^DR_UPLOAD_S3_BUCKET=\)\(.*\)/\1$DR_UPLOAD_S3_BUCKET/" system.env

############## 모델 목록 확인 ##############
aws s3 ls s3://${DR_LOCAL_S3_BUCKET}

```

### 모델 설정 1

```bash
############## 모델 설정 ############

# 모델명 (폴더명)
DR_LOCAL_S3_MODEL_PREFIX=model_name

# Clone 여부
DR_LOCAL_S3_PRETRAINED=False  

# Clone 할 모델명 (폴더명)
DR_LOCAL_S3_PRETRAINED_PREFIX=pretrained_model_name 


################ 기본 Worker 설정 ################
# 트랙명
DR_WORLD_NAME=jyllandsringen_pro_ccw 

# 레이싱 유형
DR_RACE_TYPE=TIME_TRIAL 

# 순환 간격 비율 (워커별 에피소드 수를 곱했을 때 정수가 나오도록 해야 함)
DR_TRAIN_ROUND_ROBIN_ADVANCE_DIST=0.1  

# 조명 순환
DR_ENABLE_DOMAIN_RANDOMIZATION=False 

# 시계방향 순환
DR_TRAIN_ALTERNATE_DRIVING_DIRECTION=False


################ 다중 작업 설정 ################
# 서브 Worker 따로 환경 설정할 지 여부
DR_TRAIN_MULTI_CONFIG=False 

# 동시 작업 워커 수 (로보메이커 수)
DR_WORKERS=1

# 다중 작업 구성 (compose or swarm)
DR_DOCKER_STYLE=compose

###### 환경변수 등록 #####
sed -i "s/\(^DR_LOCAL_S3_MODEL_PREFIX=\)\(.*\)/\1$DR_LOCAL_S3_MODEL_PREFIX/" run.env
sed -i "s/\(^DR_LOCAL_S3_PRETRAINED=\)\(.*\)/\1$DR_LOCAL_S3_PRETRAINED/" run.env
sed -i "s/\(^DR_LOCAL_S3_PRETRAINED_PREFIX=\)\(.*\)/\1$DR_LOCAL_S3_PRETRAINED_PREFIX/" run.env
sed -i "s/\(^DR_TRAIN_MIN_EVAL_TRIALS=\)\(.*\)/\1$DR_TRAIN_MIN_EVAL_TRIALS/" run.env
sed -i "s/\(^DR_WORKERS=\)\(.*\)/\1$DR_WORKERS/" system.env
sed -i "s/\(^DR_DOCKER_STYLE=\)\(.*\)/\1$DR_DOCKER_STYLE/" system.env
sed -i "s/\(^DR_TRAIN_MULTI_CONFIG=\)\(.*\)/\1$DR_TRAIN_MULTI_CONFIG/" run.env
sed -i "s/\(^DR_WORLD_NAME=\)\(.*\)/\1$DR_WORLD_NAME/" run.env
sed -i "s/\(^DR_RACE_TYPE=\)\(.*\)/\1$DR_RACE_TYPE/" run.env
sed -i "s/\(^DR_ENABLE_DOMAIN_RANDOMIZATION=\)\(.*\)/\1$DR_ENABLE_DOMAIN_RANDOMIZATION/" run.env
sed -i "s/\(^DR_TRAIN_ALTERNATE_DRIVING_DIRECTION=\)\(.*\)/\1$DR_TRAIN_ALTERNATE_DRIVING_DIRECTION/" run.env
sed -i "s/\(^DR_TRAIN_ROUND_ROBIN_ADVANCE_DIST=\)\(.*\)/\1$DR_TRAIN_ROUND_ROBIN_ADVANCE_DIST/" run.env

```

### 모델 설정 2

```bash
############# model_metadata.json ###########
cat <<EOF >./custom_files/model_metadata.json
{
  "action_space" : [
    {"steering_angle":0.0,"speed":4.0,"index":0},
    {"steering_angle":0.0,"speed":3.5,"index":1},
    {"steering_angle":6.0,"speed":3.5,"index":2},
    {"steering_angle":-6.0,"speed":3.5,"index":3},
    {"steering_angle":12.0,"speed":3.0,"index":4},
    {"steering_angle":-12.0,"speed":3.0,"index":5},
    {"steering_angle":18.0,"speed":2.5,"index":6},
    {"steering_angle":-18.0,"speed":2.5,"index":7},
    {"steering_angle":24.0,"speed":2.0,"index":8},
    {"steering_angle":-24.0,"speed":2.0,"index":9},
    {"steering_angle":30.0,"speed":1.5,"index":10},
    {"steering_angle":-30.0,"speed":1.5,"index":11}
  ],
  "sensor": ["FRONT_FACING_CAMERA"],
  "neural_network": "DEEP_CONVOLUTIONAL_NETWORK_SHALLOW",
  "training_algorithm": "clipped_ppo", 
  "action_space_type": "discrete",
  "version": "4"
}
EOF

############## reward_function.py ###############
cat <<EOF >./custom_files/reward_function.py
def reward_function(params):
    track_length = params['track_length']
    steps = params['steps']
    progress = params['progress']
    global progress_prev
    if steps <= 2 : 
        progress_prev = progress

    delta_progress = progress - progress_prev
    delta_distance = track_length * (delta_progress/100)
    delta_distance = max(min(delta_distance,1),-1)
    
    reward = delta_distance**3
    progress_prev = progress  
    return float(reward)
EOF

############# hyperparameters.json ############
cat <<EOF >./custom_files/hyperparameters.json
{
    "batch_size": 64,
    "beta_entropy": 0.01,
    "discount_factor": 0.99,
    "e_greedy_value": 0.05,
    "epsilon_steps": 100000,
    "exploration_type": "categorical",
    "loss_type": "huber",
    "lr": 0.0003,
    "num_episodes_between_training": 30,
    "num_epochs": 10,
    "stack_size": 1,
    "term_cond_avg_score": 100000.0,
    "term_cond_max_episodes": 100000,
    "sac_alpha": 0.2
  }
EOF


```

## 2. Worker 3개

### 루트 사용자 전환
  
```bash
sudo su

```

### 버킷 설정
  
```bash
# 작업폴더이동
cd ~/deepracer-for-cloud/

# dr-* 명령어 가져오기
source bin/activate.sh

############## 주요 환경변수 설정 ##############
# S3 버킷 - 로컬
DR_LOCAL_S3_BUCKET=drfc-....-local

# S3 버킷 - 업로드
DR_UPLOAD_S3_BUCKET=drfc-....-upload

# 버킷 생성
aws s3 mb s3://${DR_LOCAL_S3_BUCKET} --region us-east-1
aws s3 mb s3://${DR_UPLOAD_S3_BUCKET} --region us-east-1

# 환경변수 등록
sed -i "s/\(^DR_LOCAL_S3_BUCKET=\)\(.*\)/\1$DR_LOCAL_S3_BUCKET/" system.env
sed -i "s/\(^DR_UPLOAD_S3_BUCKET=\)\(.*\)/\1$DR_UPLOAD_S3_BUCKET/" system.env

############## 모델 목록 확인 ##############
aws s3 ls s3://${DR_LOCAL_S3_BUCKET}

```

### 모델 설정 1

```bash
############## 모델 설정 ############

# 모델명 (폴더명)
DR_LOCAL_S3_MODEL_PREFIX=model_name

# Clone 여부
DR_LOCAL_S3_PRETRAINED=False  

# Clone 할 모델명 (폴더명)
DR_LOCAL_S3_PRETRAINED_PREFIX=pretrained_model_name 


################ 기본 Worker 설정 ################
# 트랙명
DR_WORLD_NAME=jyllandsringen_pro_ccw 

# 레이싱 유형
DR_RACE_TYPE=TIME_TRIAL 

# 순환 간격 비율 (워커별 에피소드 수를 곱했을 때 정수가 나오도록 해야 함)
DR_TRAIN_ROUND_ROBIN_ADVANCE_DIST=0.1  

# 조명 순환
DR_ENABLE_DOMAIN_RANDOMIZATION=False 

# 시계방향 순환
DR_TRAIN_ALTERNATE_DRIVING_DIRECTION=False


################ 다중 작업 설정 ################
# 서브 Worker 따로 환경 설정할 지 여부
DR_TRAIN_MULTI_CONFIG=False 

# 동시 작업 워커 수 (로보메이커 수)
DR_WORKERS=3

# 다중 작업 구성 (compose or swarm)
DR_DOCKER_STYLE=compose

###### 환경변수 등록 #####
sed -i "s/\(^DR_LOCAL_S3_MODEL_PREFIX=\)\(.*\)/\1$DR_LOCAL_S3_MODEL_PREFIX/" run.env
sed -i "s/\(^DR_LOCAL_S3_PRETRAINED=\)\(.*\)/\1$DR_LOCAL_S3_PRETRAINED/" run.env
sed -i "s/\(^DR_LOCAL_S3_PRETRAINED_PREFIX=\)\(.*\)/\1$DR_LOCAL_S3_PRETRAINED_PREFIX/" run.env
sed -i "s/\(^DR_TRAIN_MIN_EVAL_TRIALS=\)\(.*\)/\1$DR_TRAIN_MIN_EVAL_TRIALS/" run.env
sed -i "s/\(^DR_WORKERS=\)\(.*\)/\1$DR_WORKERS/" system.env
sed -i "s/\(^DR_DOCKER_STYLE=\)\(.*\)/\1$DR_DOCKER_STYLE/" system.env
sed -i "s/\(^DR_TRAIN_MULTI_CONFIG=\)\(.*\)/\1$DR_TRAIN_MULTI_CONFIG/" run.env
sed -i "s/\(^DR_WORLD_NAME=\)\(.*\)/\1$DR_WORLD_NAME/" run.env
sed -i "s/\(^DR_RACE_TYPE=\)\(.*\)/\1$DR_RACE_TYPE/" run.env
sed -i "s/\(^DR_ENABLE_DOMAIN_RANDOMIZATION=\)\(.*\)/\1$DR_ENABLE_DOMAIN_RANDOMIZATION/" run.env
sed -i "s/\(^DR_TRAIN_ALTERNATE_DRIVING_DIRECTION=\)\(.*\)/\1$DR_TRAIN_ALTERNATE_DRIVING_DIRECTION/" run.env
sed -i "s/\(^DR_TRAIN_ROUND_ROBIN_ADVANCE_DIST=\)\(.*\)/\1$DR_TRAIN_ROUND_ROBIN_ADVANCE_DIST/" run.env

```

### 모델 설정 2

```bash
############# model_metadata.json ###########
cat <<EOF >./custom_files/model_metadata.json
{
  "action_space" : [
    {"steering_angle":0.0,"speed":4.0,"index":0},
    {"steering_angle":0.0,"speed":3.5,"index":1},
    {"steering_angle":6.0,"speed":3.5,"index":2},
    {"steering_angle":-6.0,"speed":3.5,"index":3},
    {"steering_angle":12.0,"speed":3.0,"index":4},
    {"steering_angle":-12.0,"speed":3.0,"index":5},
    {"steering_angle":18.0,"speed":2.5,"index":6},
    {"steering_angle":-18.0,"speed":2.5,"index":7},
    {"steering_angle":24.0,"speed":2.0,"index":8},
    {"steering_angle":-24.0,"speed":2.0,"index":9},
    {"steering_angle":30.0,"speed":1.5,"index":10},
    {"steering_angle":-30.0,"speed":1.5,"index":11}
  ],
  "sensor": ["FRONT_FACING_CAMERA"],
  "neural_network": "DEEP_CONVOLUTIONAL_NETWORK_SHALLOW",
  "training_algorithm": "clipped_ppo", 
  "action_space_type": "discrete",
  "version": "4"
}
EOF

############## reward_function.py ###############
cat <<EOF >./custom_files/reward_function.py
def reward_function(params):
    track_length = params['track_length']
    steps = params['steps']
    progress = params['progress']
    global progress_prev
    if steps <= 2 : 
        progress_prev = progress

    delta_progress = progress - progress_prev
    delta_distance = track_length * (delta_progress/100)
    delta_distance = max(min(delta_distance,1),-1)
    
    reward = delta_distance**3
    progress_prev = progress  
    return float(reward)
EOF

############# hyperparameters.json ############
cat <<EOF >./custom_files/hyperparameters.json
{
    "batch_size": 64,
    "beta_entropy": 0.01,
    "discount_factor": 0.99,
    "e_greedy_value": 0.05,
    "epsilon_steps": 150000,
    "exploration_type": "categorical",
    "loss_type": "huber",
    "lr": 0.0003,
    "num_episodes_between_training": 60,
    "num_epochs": 7,
    "stack_size": 1,
    "term_cond_avg_score": 100000.0,
    "term_cond_max_episodes": 150000,
    "sac_alpha": 0.2
  }
EOF


```

## 3. Worker 8개

### 루트 사용자 전환
  
```bash
sudo su

```

### 버킷 설정
  
```bash
# 작업폴더이동
cd ~/deepracer-for-cloud/

# dr-* 명령어 가져오기
source bin/activate.sh

############## 주요 환경변수 설정 ##############
# S3 버킷 - 로컬
DR_LOCAL_S3_BUCKET=drfc-....-local

# S3 버킷 - 업로드
DR_UPLOAD_S3_BUCKET=drfc-....-upload

# 버킷 생성
aws s3 mb s3://${DR_LOCAL_S3_BUCKET} --region us-east-1
aws s3 mb s3://${DR_UPLOAD_S3_BUCKET} --region us-east-1

# 환경변수 등록
sed -i "s/\(^DR_LOCAL_S3_BUCKET=\)\(.*\)/\1$DR_LOCAL_S3_BUCKET/" system.env
sed -i "s/\(^DR_UPLOAD_S3_BUCKET=\)\(.*\)/\1$DR_UPLOAD_S3_BUCKET/" system.env

############## 모델 목록 확인 ##############
aws s3 ls s3://${DR_LOCAL_S3_BUCKET}

```

### 모델 설정 1

```bash
############## 모델 설정 ############

# 모델명 (폴더명)
DR_LOCAL_S3_MODEL_PREFIX=model_name

# Clone 여부
DR_LOCAL_S3_PRETRAINED=False  

# Clone 할 모델명 (폴더명)
DR_LOCAL_S3_PRETRAINED_PREFIX=pretrained_model_name 


################ 기본 Worker 설정 ################
# 트랙명
DR_WORLD_NAME=jyllandsringen_pro_ccw 

# 레이싱 유형
DR_RACE_TYPE=TIME_TRIAL 

# 순환 간격 비율 (워커별 에피소드 수를 곱했을 때 정수가 나오도록 해야 함)
DR_TRAIN_ROUND_ROBIN_ADVANCE_DIST=0.1  

# 조명 순환
DR_ENABLE_DOMAIN_RANDOMIZATION=False 

# 시계방향 순환
DR_TRAIN_ALTERNATE_DRIVING_DIRECTION=False


################ 다중 작업 설정 ################
# 서브 Worker 따로 환경 설정할 지 여부
DR_TRAIN_MULTI_CONFIG=False 

# 동시 작업 워커 수 (로보메이커 수)
DR_WORKERS=8

# 다중 작업 구성 (compose or swarm)
DR_DOCKER_STYLE=compose

###### 환경변수 등록 #####
sed -i "s/\(^DR_LOCAL_S3_MODEL_PREFIX=\)\(.*\)/\1$DR_LOCAL_S3_MODEL_PREFIX/" run.env
sed -i "s/\(^DR_LOCAL_S3_PRETRAINED=\)\(.*\)/\1$DR_LOCAL_S3_PRETRAINED/" run.env
sed -i "s/\(^DR_LOCAL_S3_PRETRAINED_PREFIX=\)\(.*\)/\1$DR_LOCAL_S3_PRETRAINED_PREFIX/" run.env
sed -i "s/\(^DR_TRAIN_MIN_EVAL_TRIALS=\)\(.*\)/\1$DR_TRAIN_MIN_EVAL_TRIALS/" run.env
sed -i "s/\(^DR_WORKERS=\)\(.*\)/\1$DR_WORKERS/" system.env
sed -i "s/\(^DR_DOCKER_STYLE=\)\(.*\)/\1$DR_DOCKER_STYLE/" system.env
sed -i "s/\(^DR_TRAIN_MULTI_CONFIG=\)\(.*\)/\1$DR_TRAIN_MULTI_CONFIG/" run.env
sed -i "s/\(^DR_WORLD_NAME=\)\(.*\)/\1$DR_WORLD_NAME/" run.env
sed -i "s/\(^DR_RACE_TYPE=\)\(.*\)/\1$DR_RACE_TYPE/" run.env
sed -i "s/\(^DR_ENABLE_DOMAIN_RANDOMIZATION=\)\(.*\)/\1$DR_ENABLE_DOMAIN_RANDOMIZATION/" run.env
sed -i "s/\(^DR_TRAIN_ALTERNATE_DRIVING_DIRECTION=\)\(.*\)/\1$DR_TRAIN_ALTERNATE_DRIVING_DIRECTION/" run.env
sed -i "s/\(^DR_TRAIN_ROUND_ROBIN_ADVANCE_DIST=\)\(.*\)/\1$DR_TRAIN_ROUND_ROBIN_ADVANCE_DIST/" run.env

```

### 모델 설정 2

```bash
############# model_metadata.json ###########
cat <<EOF >./custom_files/model_metadata.json
{
  "action_space" : [
    {"steering_angle":0.0,"speed":4.0,"index":0},
    {"steering_angle":0.0,"speed":3.5,"index":1},
    {"steering_angle":6.0,"speed":3.5,"index":2},
    {"steering_angle":-6.0,"speed":3.5,"index":3},
    {"steering_angle":12.0,"speed":3.0,"index":4},
    {"steering_angle":-12.0,"speed":3.0,"index":5},
    {"steering_angle":18.0,"speed":2.5,"index":6},
    {"steering_angle":-18.0,"speed":2.5,"index":7},
    {"steering_angle":24.0,"speed":2.0,"index":8},
    {"steering_angle":-24.0,"speed":2.0,"index":9},
    {"steering_angle":30.0,"speed":1.5,"index":10},
    {"steering_angle":-30.0,"speed":1.5,"index":11}
  ],
  "sensor": ["FRONT_FACING_CAMERA"],
  "neural_network": "DEEP_CONVOLUTIONAL_NETWORK_SHALLOW",
  "training_algorithm": "clipped_ppo", 
  "action_space_type": "discrete",
  "version": "4"
}
EOF

############## reward_function.py ###############
cat <<EOF >./custom_files/reward_function.py
def reward_function(params):
    track_length = params['track_length']
    steps = params['steps']
    progress = params['progress']
    global progress_prev
    if steps <= 2 : 
        progress_prev = progress

    delta_progress = progress - progress_prev
    delta_distance = track_length * (delta_progress/100)
    delta_distance = max(min(delta_distance,1),-1)
    
    reward = delta_distance**3
    progress_prev = progress  
    return float(reward)
EOF

############# hyperparameters.json ############
cat <<EOF >./custom_files/hyperparameters.json
{
    "batch_size": 64,
    "beta_entropy": 0.01,
    "discount_factor": 0.99,
    "e_greedy_value": 0.05,
    "epsilon_steps": 200000,
    "exploration_type": "categorical",
    "loss_type": "huber",
    "lr": 0.0003,
    "num_episodes_between_training": 160,
    "num_epochs": 5,
    "stack_size": 1,
    "term_cond_avg_score": 100000.0,
    "term_cond_max_episodes": 200000,
    "sac_alpha": 0.2
  }
EOF


```

## 4. Worker 8개 - 다중 환경

### 루트 사용자 전환
  
```bash
sudo su

```

### 버킷 설정
  
```bash
# 작업폴더이동
cd ~/deepracer-for-cloud/

# dr-* 명령어 가져오기
source bin/activate.sh

############## 주요 환경변수 설정 ##############
# S3 버킷 - 로컬
DR_LOCAL_S3_BUCKET=drfc-....-local

# S3 버킷 - 업로드
DR_UPLOAD_S3_BUCKET=drfc-....-upload

# 버킷 생성
aws s3 mb s3://${DR_LOCAL_S3_BUCKET} --region us-east-1
aws s3 mb s3://${DR_UPLOAD_S3_BUCKET} --region us-east-1

# 환경변수 등록
sed -i "s/\(^DR_LOCAL_S3_BUCKET=\)\(.*\)/\1$DR_LOCAL_S3_BUCKET/" system.env
sed -i "s/\(^DR_UPLOAD_S3_BUCKET=\)\(.*\)/\1$DR_UPLOAD_S3_BUCKET/" system.env

############## 모델 목록 확인 ##############
aws s3 ls s3://${DR_LOCAL_S3_BUCKET}

```

### 모델 설정 1

```bash
############## 모델 설정 ############

# 모델명 (폴더명)
DR_LOCAL_S3_MODEL_PREFIX=model_name

# Clone 여부
DR_LOCAL_S3_PRETRAINED=False  

# Clone 할 모델명 (폴더명)
DR_LOCAL_S3_PRETRAINED_PREFIX=pretrained_model_name 


################ 기본 Worker 설정 ################
# 트랙명
DR_WORLD_NAME=reinvent_base 

# 레이싱 유형
DR_RACE_TYPE=TIME_TRIAL 

# 순환 간격 비율 (워커별 에피소드 수를 곱했을 때 정수가 나오도록 해야 함)
DR_TRAIN_ROUND_ROBIN_ADVANCE_DIST=0.1  

# 조명 순환
DR_ENABLE_DOMAIN_RANDOMIZATION=False 

# 시계방향 순환
DR_TRAIN_ALTERNATE_DRIVING_DIRECTION=False


################ 다중 작업 설정 ################
# 서브 Worker 따로 환경 설정할 지 여부
DR_TRAIN_MULTI_CONFIG=True 

# 동시 작업 워커 수 (로보메이커 수)
DR_WORKERS=8

# 다중 작업 구성 (compose or swarm)
DR_DOCKER_STYLE=compose

###### 환경변수 등록 #####
sed -i "s/\(^DR_LOCAL_S3_MODEL_PREFIX=\)\(.*\)/\1$DR_LOCAL_S3_MODEL_PREFIX/" run.env
sed -i "s/\(^DR_LOCAL_S3_PRETRAINED=\)\(.*\)/\1$DR_LOCAL_S3_PRETRAINED/" run.env
sed -i "s/\(^DR_LOCAL_S3_PRETRAINED_PREFIX=\)\(.*\)/\1$DR_LOCAL_S3_PRETRAINED_PREFIX/" run.env
sed -i "s/\(^DR_TRAIN_MIN_EVAL_TRIALS=\)\(.*\)/\1$DR_TRAIN_MIN_EVAL_TRIALS/" run.env
sed -i "s/\(^DR_WORKERS=\)\(.*\)/\1$DR_WORKERS/" system.env
sed -i "s/\(^DR_DOCKER_STYLE=\)\(.*\)/\1$DR_DOCKER_STYLE/" system.env
sed -i "s/\(^DR_TRAIN_MULTI_CONFIG=\)\(.*\)/\1$DR_TRAIN_MULTI_CONFIG/" run.env
sed -i "s/\(^DR_WORLD_NAME=\)\(.*\)/\1$DR_WORLD_NAME/" run.env
sed -i "s/\(^DR_RACE_TYPE=\)\(.*\)/\1$DR_RACE_TYPE/" run.env
sed -i "s/\(^DR_ENABLE_DOMAIN_RANDOMIZATION=\)\(.*\)/\1$DR_ENABLE_DOMAIN_RANDOMIZATION/" run.env
sed -i "s/\(^DR_TRAIN_ALTERNATE_DRIVING_DIRECTION=\)\(.*\)/\1$DR_TRAIN_ALTERNATE_DRIVING_DIRECTION/" run.env
sed -i "s/\(^DR_TRAIN_ROUND_ROBIN_ADVANCE_DIST=\)\(.*\)/\1$DR_TRAIN_ROUND_ROBIN_ADVANCE_DIST/" run.env

```

### 모델 설정 2

```bash
############# model_metadata.json ###########
cat <<EOF >./custom_files/model_metadata.json
{
  "action_space" : [
    {"steering_angle":0.0,"speed":0.7,"index":0},
    {"steering_angle":25.0,"speed":0.7,"index":1},
    {"steering_angle":-25.0,"speed":0.7,"index":2}
  ],
  "sensor": ["FRONT_FACING_CAMERA"],
  "neural_network": "DEEP_CONVOLUTIONAL_NETWORK_SHALLOW",
  "training_algorithm": "clipped_ppo", 
  "action_space_type": "discrete",
  "version": "4"
}
EOF

############## reward_function.py ###############
cat <<EOF >./custom_files/reward_function.py
def reward_function(params):
    # Read input parameters
    track_width = params['track_width']
    distance_from_center = params['distance_from_center']
    
    # Calculate 3 markers that are at varying distances away from the center line
    marker_1 = 0.1 * track_width
    marker_2 = 0.25 * track_width
    marker_3 = 0.5 * track_width
    
    # Give higher reward if the car is closer to center line and vice versa
    if distance_from_center <= marker_1:
        reward = 1.0
    elif distance_from_center <= marker_2:
        reward = 0.5
    elif distance_from_center <= marker_3:
        reward = 0.1
    else:
        reward = 1e-3  # likely crashed/ close to off track
    return float(reward)
EOF

############# hyperparameters.json ############
cat <<EOF >./custom_files/hyperparameters.json
{
    "batch_size": 64,
    "beta_entropy": 0.01,
    "discount_factor": 0.99,
    "e_greedy_value": 0.05,
    "epsilon_steps": 200000,
    "exploration_type": "categorical",
    "loss_type": "huber",
    "lr": 0.0003,
    "num_episodes_between_training": 160,
    "num_epochs": 10,
    "stack_size": 1,
    "term_cond_avg_score": 100000.0,
    "term_cond_max_episodes": 200000,
    "sac_alpha": 0.2
  }
EOF


```

### 서브 worker 설정
  
```bash
#### WORKER 2 ####
WORKER_NUM=2
cp defaults/template-worker.env worker-$WORKER_NUM.env 

DR_WORLD_NAME=reinvent_base
DR_RACE_TYPE=TIME_TRIAL
DR_ENABLE_DOMAIN_RANDOMIZATION=True
DR_TRAIN_ALTERNATE_DRIVING_DIRECTION=False

sed -i "s/\(^DR_TRAIN_MULTI_CONFIG=\)\(.*\)/\1$DR_TRAIN_MULTI_CONFIG/" worker-$WORKER_NUM.env 
sed -i "s/\(^DR_WORLD_NAME=\)\(.*\)/\1$DR_WORLD_NAME/" worker-$WORKER_NUM.env 
sed -i "s/\(^DR_RACE_TYPE=\)\(.*\)/\1$DR_RACE_TYPE/" worker-$WORKER_NUM.env 
sed -i "s/\(^DR_ENABLE_DOMAIN_RANDOMIZATION=\)\(.*\)/\1$DR_ENABLE_DOMAIN_RANDOMIZATION/" worker-$WORKER_NUM.env 
sed -i "s/\(^DR_TRAIN_ALTERNATE_DRIVING_DIRECTION=\)\(.*\)/\1$DR_TRAIN_ALTERNATE_DRIVING_DIRECTION/" worker-$WORKER_NUM.env 

#### WORKER 3 ####
WORKER_NUM=3
cp defaults/template-worker.env worker-$WORKER_NUM.env 

DR_WORLD_NAME=reInvent2019_track
DR_RACE_TYPE=TIME_TRIAL
DR_ENABLE_DOMAIN_RANDOMIZATION=True
DR_TRAIN_ALTERNATE_DRIVING_DIRECTION=False

sed -i "s/\(^DR_TRAIN_MULTI_CONFIG=\)\(.*\)/\1$DR_TRAIN_MULTI_CONFIG/" worker-$WORKER_NUM.env 
sed -i "s/\(^DR_WORLD_NAME=\)\(.*\)/\1$DR_WORLD_NAME/" worker-$WORKER_NUM.env 
sed -i "s/\(^DR_RACE_TYPE=\)\(.*\)/\1$DR_RACE_TYPE/" worker-$WORKER_NUM.env 
sed -i "s/\(^DR_ENABLE_DOMAIN_RANDOMIZATION=\)\(.*\)/\1$DR_ENABLE_DOMAIN_RANDOMIZATION/" worker-$WORKER_NUM.env 
sed -i "s/\(^DR_TRAIN_ALTERNATE_DRIVING_DIRECTION=\)\(.*\)/\1$DR_TRAIN_ALTERNATE_DRIVING_DIRECTION/" worker-$WORKER_NUM.env 

#### WORKER 4 ####
WORKER_NUM=4
cp defaults/template-worker.env worker-$WORKER_NUM.env 

DR_WORLD_NAME=reInvent2019_wide
DR_RACE_TYPE=TIME_TRIAL
DR_ENABLE_DOMAIN_RANDOMIZATION=True
DR_TRAIN_ALTERNATE_DRIVING_DIRECTION=False

sed -i "s/\(^DR_TRAIN_MULTI_CONFIG=\)\(.*\)/\1$DR_TRAIN_MULTI_CONFIG/" worker-$WORKER_NUM.env 
sed -i "s/\(^DR_WORLD_NAME=\)\(.*\)/\1$DR_WORLD_NAME/" worker-$WORKER_NUM.env 
sed -i "s/\(^DR_RACE_TYPE=\)\(.*\)/\1$DR_RACE_TYPE/" worker-$WORKER_NUM.env 
sed -i "s/\(^DR_ENABLE_DOMAIN_RANDOMIZATION=\)\(.*\)/\1$DR_ENABLE_DOMAIN_RANDOMIZATION/" worker-$WORKER_NUM.env 
sed -i "s/\(^DR_TRAIN_ALTERNATE_DRIVING_DIRECTION=\)\(.*\)/\1$DR_TRAIN_ALTERNATE_DRIVING_DIRECTION/" worker-$WORKER_NUM.env 

#### WORKER 5 ####
WORKER_NUM=5
cp defaults/template-worker.env worker-$WORKER_NUM.env 

DR_WORLD_NAME=Singapore_building
DR_RACE_TYPE=TIME_TRIAL
DR_ENABLE_DOMAIN_RANDOMIZATION=True
DR_TRAIN_ALTERNATE_DRIVING_DIRECTION=False

sed -i "s/\(^DR_TRAIN_MULTI_CONFIG=\)\(.*\)/\1$DR_TRAIN_MULTI_CONFIG/" worker-$WORKER_NUM.env 
sed -i "s/\(^DR_WORLD_NAME=\)\(.*\)/\1$DR_WORLD_NAME/" worker-$WORKER_NUM.env 
sed -i "s/\(^DR_RACE_TYPE=\)\(.*\)/\1$DR_RACE_TYPE/" worker-$WORKER_NUM.env 
sed -i "s/\(^DR_ENABLE_DOMAIN_RANDOMIZATION=\)\(.*\)/\1$DR_ENABLE_DOMAIN_RANDOMIZATION/" worker-$WORKER_NUM.env 
sed -i "s/\(^DR_TRAIN_ALTERNATE_DRIVING_DIRECTION=\)\(.*\)/\1$DR_TRAIN_ALTERNATE_DRIVING_DIRECTION/" worker-$WORKER_NUM.env 

#### WORKER 6 ####
WORKER_NUM=6
cp defaults/template-worker.env worker-$WORKER_NUM.env 

DR_WORLD_NAME=Vegas_track
DR_RACE_TYPE=TIME_TRIAL
DR_ENABLE_DOMAIN_RANDOMIZATION=True
DR_TRAIN_ALTERNATE_DRIVING_DIRECTION=False

sed -i "s/\(^DR_TRAIN_MULTI_CONFIG=\)\(.*\)/\1$DR_TRAIN_MULTI_CONFIG/" worker-$WORKER_NUM.env 
sed -i "s/\(^DR_WORLD_NAME=\)\(.*\)/\1$DR_WORLD_NAME/" worker-$WORKER_NUM.env 
sed -i "s/\(^DR_RACE_TYPE=\)\(.*\)/\1$DR_RACE_TYPE/" worker-$WORKER_NUM.env 
sed -i "s/\(^DR_ENABLE_DOMAIN_RANDOMIZATION=\)\(.*\)/\1$DR_ENABLE_DOMAIN_RANDOMIZATION/" worker-$WORKER_NUM.env 
sed -i "s/\(^DR_TRAIN_ALTERNATE_DRIVING_DIRECTION=\)\(.*\)/\1$DR_TRAIN_ALTERNATE_DRIVING_DIRECTION/" worker-$WORKER_NUM.env 

#### WORKER 7 ####
WORKER_NUM=7
cp defaults/template-worker.env worker-$WORKER_NUM.env 

DR_WORLD_NAME=China_track
DR_RACE_TYPE=TIME_TRIAL
DR_ENABLE_DOMAIN_RANDOMIZATION=True
DR_TRAIN_ALTERNATE_DRIVING_DIRECTION=False

sed -i "s/\(^DR_TRAIN_MULTI_CONFIG=\)\(.*\)/\1$DR_TRAIN_MULTI_CONFIG/" worker-$WORKER_NUM.env 
sed -i "s/\(^DR_WORLD_NAME=\)\(.*\)/\1$DR_WORLD_NAME/" worker-$WORKER_NUM.env 
sed -i "s/\(^DR_RACE_TYPE=\)\(.*\)/\1$DR_RACE_TYPE/" worker-$WORKER_NUM.env 
sed -i "s/\(^DR_ENABLE_DOMAIN_RANDOMIZATION=\)\(.*\)/\1$DR_ENABLE_DOMAIN_RANDOMIZATION/" worker-$WORKER_NUM.env 
sed -i "s/\(^DR_TRAIN_ALTERNATE_DRIVING_DIRECTION=\)\(.*\)/\1$DR_TRAIN_ALTERNATE_DRIVING_DIRECTION/" worker-$WORKER_NUM.env 

#### WORKER 8 ####
WORKER_NUM=8
cp defaults/template-worker.env worker-$WORKER_NUM.env 

DR_WORLD_NAME=caecer_gp
DR_RACE_TYPE=TIME_TRIAL
DR_ENABLE_DOMAIN_RANDOMIZATION=True
DR_TRAIN_ALTERNATE_DRIVING_DIRECTION=False

sed -i "s/\(^DR_TRAIN_MULTI_CONFIG=\)\(.*\)/\1$DR_TRAIN_MULTI_CONFIG/" worker-$WORKER_NUM.env 
sed -i "s/\(^DR_WORLD_NAME=\)\(.*\)/\1$DR_WORLD_NAME/" worker-$WORKER_NUM.env 
sed -i "s/\(^DR_RACE_TYPE=\)\(.*\)/\1$DR_RACE_TYPE/" worker-$WORKER_NUM.env 
sed -i "s/\(^DR_ENABLE_DOMAIN_RANDOMIZATION=\)\(.*\)/\1$DR_ENABLE_DOMAIN_RANDOMIZATION/" worker-$WORKER_NUM.env 
sed -i "s/\(^DR_TRAIN_ALTERNATE_DRIVING_DIRECTION=\)\(.*\)/\1$DR_TRAIN_ALTERNATE_DRIVING_DIRECTION/" worker-$WORKER_NUM.env


```
